# LOL Models

## Data Export
For LOL modeling is done for performance score because that is the per game score which
translates better across different leagues
```
# player LOL data
dumpdata.sc --seasons 2016 2017 2018 2019 2020 2021 2022 --stats "*" --progress lol_hist_2014-2022.scored.db \
    --no_teams --target_calc_stats "*performance_score" --hist_recent_games 3 \
    --hist_recent_mode ma -f lol_player.csv

# team LOL data
dumpdata.sc --seasons 2014 2015 2016 2017 2018 2019 2020 2021 2022 --stats "*" --progress \
    lol_hist_2014-2022.scored.db --no_players --target_calc_stats "*performance_score" \
    --hist_extra match-win --current_extra match-win \
    --hist_recent_games 3 --hist_recent_mode ma \
    -f lol_team.csv
```

In [ ]:
RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 2022
RECENT_GAMES = 3


In [ ]:
from datetime import datetime

from fantasy_py import PlayerOrTeam

from train_test import load_data, train_test, create_fantasy_model

TARGET = ("extra", "match_win")
MODEL_NAME = "LOL-team-win"
DATA_FILENAME = "/fantasy/lol_team.csv"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None
TRAINING_SEASONS = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

# TARGET = ("calc", "dk_performance_score")
# MODEL_NAME = "LOL-team-DK"
# DATA_FILENAME = "/fantasy/lol_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TRAINING_SEASONS = [2016, 2017, 2018, 2019, 2020, 2021]

# TARGET = ("calc", "dk_performance_score")
# MODEL_NAME = "LOL-player-DK"
# DATA_FILENAME = "/fantasy/lol_player.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True
# TRAINING_SEASONS = [2016, 2017, 2018, 2019, 2020, 2021]

df, tt_data = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    seed=RANDOM_SEED,
    include_position=INCLUDE_POS,
)

# for automl_type in ["tpot", "autosk"]:
for automl_type in ["tpot"]:
    dt_trained = datetime.now()

    model_filepath, performance = train_test(
        automl_type, MODEL_NAME, TARGET, tt_data, RANDOM_SEED, TRAINING_TIME, dt_trained
    )
    performance["season"] = VALIDATION_SEASON
    model = create_fantasy_model(
        MODEL_NAME,
        model_filepath,
        dt_trained,
        tt_data[0].columns,
        TARGET,
        TRAINING_TIME,
        P_OR_T,
        RECENT_GAMES,
        automl_type,
        performance,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
    )
    model_filepath = model.dump(".".join([MODEL_NAME, TARGET[1], automl_type, "model"]))
    print(f"Model file saved to '{model_filepath}'")
